# Egocentric Video Mapper

Before starting make sure you have an available GPU. If you are unsure about whether you have an available GPU or if you want to check which GPU you will be working with run the next cell.

In [2]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-72b7ddec-7074-1583-0076-9281c065ae9d)


This next cell will import the modules and functions that will be used for this Alpha Lab

In [3]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from utils import (
    VideoHandler,
    write_timestamp_csv,
    generate_mapper_kwargs,
    generate_comparison_video_kwargs,
)
from optic_flow import OpticFlowCalculatorLK
from sync_videos import OffsetCalculator
from gaze_mapper import RulesBasedGazeMapper
from video_renderer import save_comparison_video, save_gaze_video

Fill out the path to the directory of the uncompressed Scene Video + Timeseries download from Cloud.

In [4]:
neon_timeseries_path = "Path/To/NeonTimeSeriesFolder"
neon_vid_path = Path(neon_timeseries_path).rglob("*.mp4").__next__()

Fill out the path of the corresponding alternative egocentric view recording (please make sure that both videos have the same orientation).

In [5]:
alternative_vid_path = "Path/To/AlternativeVideo"

Define an output directory where the different output files will be saved.

In [6]:
output_dir = "Path/To/OutputFolder"

## Step 1: Video Synchronization

In this first step, optic flow in both videos will be calculated. Each optic flow result is saved to a csv file in an automatically created optic_flow subdirectory in the output directory. The csv file will contain the following columns:
- start: the start frame timestamp of the optic flow calculation
- end: the end frame timestamp of the optic flow calculation
- dx: Average horizontal displacement  
- dy: Average vertical displacement 
- angle: Average angular change in degrees

Optic flow is then used to obtain the time offset of the Action video with respect to the Neon Scene video. 



In [7]:
neon_of = OpticFlowCalculatorLK(video_dir=neon_vid_path)
neon_of_result = neon_of.process_video(
    output_file=Path(output_dir, "optic_flow/neon_lk_of.csv")
)

alternative_of = OpticFlowCalculatorLK(video_dir=alternative_vid_path)
alternative_of_result = alternative_of.process_video(
    output_file=Path(output_dir, "optic_flow/alternative_lk_of.csv")
)

Once optic flow is measured, the time offset can be calculated and a action_worldtimestamps.csv will be generated with a similar format as Neon's world_timestamps.csv.

In [8]:
offset_calc = OffsetCalculator(
    src=alternative_of_result["dy"].values,
    src_timestamps=alternative_of_result["start"].values,
    dst=neon_of_result["dy"].values,
    dst_timestamps=neon_of_result["start"].values,
    resampling_frequency=500,
)

t_offset, pearson_corr = offset_calc.estimate_time_offset()
print(
    f"Estimated offset of alternative video with respect to Neon scene video: {t_offset} seconds (Pearson correlation: {pearson_corr})"
)

write_timestamp_csv(
    neon_timeseries_path,
    VideoHandler(alternative_vid_path).timestamps + t_offset,
    saving_path=output_dir,
)

Last timestamp of alternative camera recording (2024-05-23 14:48:20.852368384) is after the last timestamp of Neon Scene recording (2024-05-23 14:48:18.721666666)


Estimated offset of alternative video with respect to Neon scene video: 2.016 seconds (Pearson correlation: 0.9192123717590794)


## Step 2: Map gaze data to the action video

After synchronizing both videos, it is time to obtain Neon gaze signal in the coordinate system of the other video. 


<div class="alert alert-block alert-info">
Here we make use of deep learning methods to guide the gaze mapping. By default, this notebook uses an implementation of Efficient LOFTR. We have implemented other algorithms: 'LOFTR', 'DISK_LightGlue', 'DeDoDe_LightGlue'. If you wish to try them out, just write the name of the desired algorithm in the <b>image_matcher_choice</b> variable found in the cell below.
</div>

In [9]:
image_matcher_choice = "Efficient_LOFTR"  # Options: 'Efficient_LOFTR', 'LOFTR', 'DISK_LightGlue', 'DeDoDe_LightGlue'

mapper_kwargs = generate_mapper_kwargs(
    neon_timeseries_path, alternative_vid_path, output_dir, image_matcher_choice
)

mapper = RulesBasedGazeMapper(**mapper_kwargs)

Before running the mapping pipeline you can adjust how often (in seconds) you want the image correspondances refreshed. 


- The higher you set the time threshold the less time the mapping will take, however it may decrease the accuracy of the mapping. 
- The lower the time threshold the more time the mapping will take. 

<div class="alert alert-block alert-warning">

- If you leave the value at 'None' then for every gaze new correspondences will be obtained. 
- We recommend using values smaller than 1 second.
</div>
 

In [10]:
time_threshold = 0.5

In [11]:
mapping_kwargs = {
    "saving_path": Path(
        output_dir,
        f"mapped_gaze/{image_matcher_choice.lower()}/alternative_camera_gaze_lk.csv",
    ),
    "refresh_time_thrshld": time_threshold,
}
mapped_gaze_path = mapper.map_gaze(**mapping_kwargs)

Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray


0 Step through video neon(neon=41, action=1)
0 Step through video action(neon=41, action=1)
2 Step through video action(neon=41, action=2)
4 Step through video neon(neon=42, action=2)


Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray


9 Step through video action(neon=42, action=3)
10 Step through video neon(neon=44, action=5)
10 Step through video action(neon=44, action=5)
12 Step through video action(neon=44, action=6)
17 Step through video neon(neon=45, action=6)
18 Step through video action(neon=45, action=7)


Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray
Neon frame is all gray


...

7886 Step through video action(neon=1222, action=1188)
Moving keypoints1 to 39.63959884643555
7887 Step through video neon(neon=1223, action=1188)
Moving keypoints0 to 41.661312103271484
7893 Step through video action(neon=1223, action=1189)
Moving keypoints1 to 39.67296600341797
7894 Step through video neon(neon=1224, action=1189)
Moving keypoints0 to 41.69465637207031
7900 Step through video action(neon=1224, action=1190)
Moving keypoints1 to 39.70633316040039
7901 Step through video neon(neon=1225, action=1190)
Moving keypoints0 to 41.72800064086914
7906 Step through video action(neon=1225, action=1191)
Moving keypoints1 to 39.73970031738281
7907 Step through video neon(neon=1226, action=1191)
Moving keypoints0 to 41.76133346557617
7913 Step through video action(neon=1226, action=1192)
Moving keypoints1 to 39.773067474365234
7914 Step through video neon(neon=1227, action=1192)
Moving keypoints0 to 41.794677734375
7920 Step through video action(neon=1227, action=1193)
Moving keypoint

Matcher was called at 1716475697890355454 (2988 correspondences)


7986 Step through video action(neon=1237, action=1203)
Moving keypoints1 to 40.140098571777344
7987 Step through video neon(neon=1238, action=1203)
Moving keypoints0 to 42.16144561767578
7993 Step through video action(neon=1238, action=1204)
Moving keypoints1 to 40.173465728759766
7994 Step through video neon(neon=1239, action=1204)
Moving keypoints0 to 42.19478988647461
8000 Step through video action(neon=1239, action=1205)
Moving keypoints1 to 40.20683288574219
8001 Step through video neon(neon=1240, action=1205)
Moving keypoints0 to 42.22813415527344
8006 Step through video action(neon=1240, action=1206)
Moving keypoints1 to 40.24020004272461
8007 Step through video neon(neon=1241, action=1206)
Moving keypoints0 to 42.261478424072266
8013 Step through video action(neon=1241, action=1207)
Moving keypoints1 to 40.27356719970703
8014 Step through video neon(neon=1242, action=1207)
Moving keypoints0 to 42.2948112487793
8020 Step through video action(neon=1242, action=1208)
Moving keypoi

Matcher was called at 1716475698390730454 (3050 correspondences)


Refreshing transformation after 100.0 gazes
8086 Step through video action(neon=1252, action=1218)
Moving keypoints1 to 40.64059829711914
8087 Step through video neon(neon=1253, action=1218)
Moving keypoints0 to 42.661590576171875
8093 Step through video action(neon=1253, action=1219)
Moving keypoints1 to 40.67396545410156
8094 Step through video neon(neon=1254, action=1219)
Moving keypoints0 to 42.694923400878906
8100 Step through video action(neon=1254, action=1220)
Moving keypoints1 to 40.707332611083984
8101 Step through video neon(neon=1255, action=1220)
Moving keypoints0 to 42.728267669677734
8106 Step through video action(neon=1255, action=1221)
Moving keypoints1 to 40.740699768066406
8107 Step through video neon(neon=1256, action=1221)
Moving keypoints0 to 42.76161193847656
8113 Step through video action(neon=1256, action=1222)
Moving keypoints1 to 40.77406692504883
8114 Step through video neon(neon=1257, action=1222)
Moving keypoints0 to 42.79495620727539
8120 Step through vid

## Step 3 (OPTIONAL): Create visualization videos

For visualization purposes you can get a side-to-side rendering with gaze of the Neon Scene video and the alternative egocentric video.  


In [12]:
comparison_video_kwargs = generate_comparison_video_kwargs(
    neon_timeseries_path,
    alternative_vid_path,
    mapped_gaze_path,
    output_dir,
    image_matcher_choice,
)
save_comparison_video(**comparison_video_kwargs)

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Saving video at /users/sof/action_map_experiments/s_try/rendered_videos/neon_comparison_efficient_loftr_lk.mp4
Video width: 3520, Video height: 1200


Additionally, you can get only the alternative egocentric video (at its original frame rate) with the overlaid gaze circle.

In [14]:
gaze_video_args = {
    "video_path": alternative_vid_path,
    "timestamps_path": Path(output_dir, "alternative_camera_timestamps.csv"),
    "gaze_path": Path(mapped_gaze_path),
    "save_video_path": Path(
        output_dir, f"rendered_videos/{image_matcher_choice}_lk.avi"
    ),
}

save_gaze_video(**gaze_video_args)

Saving video at /users/sof/action_map_experiments/s_try/rendered_videos/Efficient_LOFTR_lk.avi
Video saved at /users/sof/action_map_experiments/s_try/rendered_videos/Efficient_LOFTR_lk.avi
